<h1>NAKB data</h1>


In [1]:
import pandas as pd

import time
from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

In [3]:

NAKB_riboswitch = pd.read_csv('NAKB_riboswitch_RNA.csv')
riboswitch_pdbids = list(NAKB_riboswitch['ID'])

NAKB_rnaaptamer = pd.read_csv('NAKB_aptamer_RNA.csv')
rnaaptamer_pdbids = list(NAKB_rnaaptamer['ID'])

ribocentre = pd.read_csv('RIBOCENTRE.csv')
ribocentre_pdbids = list(ribocentre['PDB'])

combinedrna = riboswitch_pdbids+rnaaptamer_pdbids+ribocentre_pdbids
combinedrna = set(combinedrna)
len(combinedrna)

556

In [6]:
NAKB_pdbids = []
for pdbid in combinedrna:
    if pdbid not in set(ribocentre_pdbids):
        NAKB_pdbids.append(pdbid)

len(NAKB_pdbids)


433

In [62]:
#PDB ID to sequence and secondary structure using 3DNA DSSR via RNApdbee web-server
def pdbid_2_2d_canonical(pdbid): 

    driver = webdriver.Chrome()
    #driver.implicitly_wait(30)
    
    # Open web-server
    driver.get('http://rnapdbee.cs.put.poznan.pl/')  # Replace with your target URL

    # type pdbid 
    input_pdb = driver.find_element(By.ID, "pdbId")
    input_pdb.send_keys(pdbid)

    #click Get button
    getbutton = driver.find_element(By.XPATH, "//input[@type='button' and @value='Get' and @onclick='fetchPdb()']")
    getbutton.click()

    
    time.sleep(10)

    # # pasepair identifier
    # identifier = driver.find_element(By.ID, 'fr3d')
    # identifier.click()

    #select non-canonical inclusion
    # noncancheck = driver.find_element(By.ID, "nonCanonicalHandling2")
    # noncancheck.click()
    # analyzehelices = driver.find_element(By.ID, "analyzeHelices")
    # analyzehelices.click()

    #click Run button
    runbutton = driver.find_element(By.ID, 'commitPdb')
    runbutton.click()

    # Wait for the page to load 
    time.sleep(15)

    # Get the resulting HTML
    result_html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(result_html, "html.parser")
    raw_symbols = soup.findAll('span', class_='mono')
    
    syms = []

    # #print(*raw_symbols)
    for s in raw_symbols:
        syms.append(''.join(s.get_text().split()))


    # Инициализация переменных для хранения позиций и количества
    positions = []
    count = 0

    # # Поиск подстроки 'strand' в каждой строке
    # for index, line in enumerate(syms):
    #     if 'strand' in line():  # Используем lower() для нечувствительности к регистру
    #         count += 1
    #         positions.append(index)

    for j in range(len(syms)):
        if syms[j].find('strand') != -1:
            count+=1
            positions.append(j)

    positions.append(len(syms))

    resultlist = []
    for n_strand in range(count):
        strand = syms[positions[n_strand]]
        slice = syms[positions[n_strand]:positions[n_strand+1]]
        sequence = ''
        secondary = ''
        for ss in slice:
            if ss in ('a', 't', 'g', 'c', 'u', 'A', 'T', 'G', 'C', 'U'):
                sequence = sequence+ss
            elif ss in ('(', ')', '.', '[', ']', '{', '}', '-', '<', '>'):
                secondary = secondary+ss

        resultlist.append([pdbid, strand, sequence, secondary])
    
    return resultlist



    #print(syms)

    # print(type(syms))

    
    # for ss in syms:
    #     if ss in ('a', 't', 'g', 'c', 'u', 'A', 'T', 'G', 'C', 'U'):
    #         sequence = sequence+ss
    #     elif ss in ('(', ')', '.', '[', ']', '{', '}', '-', '<', '>'):
    #         secondary = secondary+ss

    # # return [pdbid, sequence, secondary]


In [14]:
rnapdbids = NAKB_pdbids
rna_aptamers_predflist = []

for pdbid in rnapdbids:
    #rna_aptamers_predflist.append(pdbid_2_2d_noncanonical(pdbid))
    rna_aptamers_predflist = rna_aptamers_predflist + pdbid_2_2d_canonical(pdbid)
    print(len(rna_aptamers_predflist), end='-')



2-6-7-9-12-15-17-19-20-22-26-27-28-

ElementClickInterceptedException: Message: element click intercepted: Element <input type="submit" name="commit" id="commitPdb" class="run" value="Run" onclick="return validateForm('#pdbContentInput');" disabled="disabled"> is not clickable at point (387, 874). Other element would receive the click: <div class="blockUI blockOverlay" style="z-index: 1000; border: none; margin: 0px; padding: 0px; width: 100%; height: 100%; top: 0px; left: 0px; background-color: rgb(0, 0, 0); opacity: 0.6; cursor: wait; position: absolute;"></div>
  (Session info: chrome=129.0.6668.58)
Stacktrace:
#0 0x565267746b9a <unknown>
#1 0x56526742c670 <unknown>
#2 0x5652674828c6 <unknown>
#3 0x5652674807e6 <unknown>
#4 0x56526747e297 <unknown>
#5 0x56526747d6a6 <unknown>
#6 0x565267470ae8 <unknown>
#7 0x5652674a0b82 <unknown>
#8 0x5652674704c8 <unknown>
#9 0x5652674a0d4e <unknown>
#10 0x5652674bfdd6 <unknown>
#11 0x5652674a0923 <unknown>
#12 0x56526746e6e7 <unknown>
#13 0x56526746f6de <unknown>
#14 0x56526771066b 

In [16]:
'2-6-7-9-12-15-17-19-20-22-26-27-28-'.count('-')

13

In [ ]:
for pdbid in rnapdbids[13:]:
    #rna_aptamers_predflist.append(pdbid_2_2d_noncanonical(pdbid))
    rna_aptamers_predflist = rna_aptamers_predflist + pdbid_2_2d_canonical(pdbid)
    print(len(rna_aptamers_predflist), end='-')

In [20]:
for i in range(len(rnapdbids)):
    if rnapdbids[i] == '6WLM':
        print(i)

107


In [22]:
for pdbid in rnapdbids[107:]:
    #rna_aptamers_predflist.append(pdbid_2_2d_noncanonical(pdbid))
    rna_aptamers_predflist = rna_aptamers_predflist + pdbid_2_2d_canonical(pdbid)
    print(len(rna_aptamers_predflist), end='-')

182-182-183-184-187-188-190-192-194-197-199-201-202-203-208-209-211-212-213-217-218-221-223-224-225-227-228-229-230-234-236-237-240-241-242-244-246-247-248-249-251-252-253-254-256-258-260-261-262-265-267-268-271-272-275-276-278-281-282-285-286-288-289-292-293-296-297-303-304-306-307-307-310-314-315-316-319-320-322-324-326-327-328-329-331-333-335-336-337-339-341-342-344-345-346-347-348-350-351-353-358-359-362-364-364-366-368-371-372-372-374-375-378-379-380-382-384-386-388-391-393-394-396-

ElementClickInterceptedException: Message: element click intercepted: Element <input type="submit" name="commit" id="commitPdb" class="run" value="Run" onclick="return validateForm('#pdbContentInput');"> is not clickable at point (387, 874). Other element would receive the click: <div class="blockUI blockOverlay" style="z-index: 1000; border: none; margin: 0px; padding: 0px; width: 100%; height: 100%; top: 0px; left: 0px; background-color: rgb(0, 0, 0); opacity: 0.6; cursor: default; position: absolute;"></div>
  (Session info: chrome=129.0.6668.58)
Stacktrace:
#0 0x55eeb1caeb9a <unknown>
#1 0x55eeb1994670 <unknown>
#2 0x55eeb19ea8c6 <unknown>
#3 0x55eeb19e87e6 <unknown>
#4 0x55eeb19e6297 <unknown>
#5 0x55eeb19e56a6 <unknown>
#6 0x55eeb19d8ae8 <unknown>
#7 0x55eeb1a08b82 <unknown>
#8 0x55eeb19d84c8 <unknown>
#9 0x55eeb1a08d4e <unknown>
#10 0x55eeb1a27dd6 <unknown>
#11 0x55eeb1a08923 <unknown>
#12 0x55eeb19d66e7 <unknown>
#13 0x55eeb19d76de <unknown>
#14 0x55eeb1c7866b <unknown>
#15 0x5

In [23]:
pdbid

'6WLU'

In [24]:
for i in range(len(rnapdbids)):
    if rnapdbids[i] == '6WLU':
        print(i)

230


In [25]:
for pdbid in rnapdbids[230:]:
    #rna_aptamers_predflist.append(pdbid_2_2d_noncanonical(pdbid))
    rna_aptamers_predflist = rna_aptamers_predflist + pdbid_2_2d_canonical(pdbid)
    print(len(rna_aptamers_predflist), end='-')

397-399-401-401-402-403-404-407-408-411-412-413-414-416-417-418-420-421-423-425-427-428-430-431-433-435-437-438-439-441-442-444-445-447-449-450-452-455-456-457-459-460-462-463-467-468-470-471-473-476-477-479-481-483-486-

ElementClickInterceptedException: Message: element click intercepted: Element <input type="submit" name="commit" id="commitPdb" class="run" value="Run" onclick="return validateForm('#pdbContentInput');" disabled="disabled"> is not clickable at point (387, 874). Other element would receive the click: <div class="blockUI blockOverlay" style="z-index: 1000; border: none; margin: 0px; padding: 0px; width: 100%; height: 100%; top: 0px; left: 0px; background-color: rgb(0, 0, 0); opacity: 0.6; cursor: wait; position: absolute;"></div>
  (Session info: chrome=129.0.6668.58)
Stacktrace:
#0 0x55a5f5994b9a <unknown>
#1 0x55a5f567a670 <unknown>
#2 0x55a5f56d08c6 <unknown>
#3 0x55a5f56ce7e6 <unknown>
#4 0x55a5f56cc297 <unknown>
#5 0x55a5f56cb6a6 <unknown>
#6 0x55a5f56beae8 <unknown>
#7 0x55a5f56eeb82 <unknown>
#8 0x55a5f56be4c8 <unknown>
#9 0x55a5f56eed4e <unknown>
#10 0x55a5f570ddd6 <unknown>
#11 0x55a5f56ee923 <unknown>
#12 0x55a5f56bc6e7 <unknown>
#13 0x55a5f56bd6de <unknown>
#14 0x55a5f595e66b 

In [27]:
pdbid

'3SKW'

In [30]:
for pdbid in rnapdbids[285:]:
    #rna_aptamers_predflist.append(pdbid_2_2d_noncanonical(pdbid))
    rna_aptamers_predflist = rna_aptamers_predflist + pdbid_2_2d_canonical(pdbid)
    print(len(rna_aptamers_predflist), end='-')

489-490-491-492-494-497-498-500-501-503-505-508-509-511-513-514-515-518-520-525-526-527-528-529-531-533-534-536-

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


538-540-541-543-545-547-548-549-550-551-553-555-557-559-561-562-563-564-567-569-570-572-573-575-576-577-579-581-582-585-586-589-591-592-595-596-597-599-602-604-605-606-610-612-613-615-616-617-619-621-622-623-625-626-627-630-631-633-635-637-638-639-642-643-645-649-650-654-656-657-660-662-663-664-666-668-671-673-674-677-679-680-684-685-686-687-688-690-695-696-697-699-701-702-703-704-705-707-708-711-712-714-717-719-720-722-723-725-727-729-730-731-734-736-737-738-739-740-742-744-

In [29]:
for i in range(len(rnapdbids)):
    if rnapdbids[i] == '3SKW':
        print(i)

285


In [31]:
rna_aptamers_predflist

[['8GXC',
  '>strand_A',
  'gGAGCGUUACGUCCGAAAGUCGCAUUGCACUCCGCGACACGGCUCUUUAAAAACAAAAGGA',
  '((((((...[[[[[....(((((..........))))).).)))))............]]]'],
 ['8GXC',
  '>strand_B',
  'gGAGCGUUACGUCCGAAAGUCGCAUUGCACUCCGCGACACGGCUCUUUAAAAACAAAAGGA',
  '((((((...]][[[....(((((..........))))).).)))))............]]]'],
 ['9BUN',
  '>strand_A',
  'GGACUCGGAAACGUGAAGGAGAGGCGCAAGGUUAACCGCCUCAGUCCA',
  '(((((((....))([...[)((((((]........]))))))))))).'],
 ['9BUN',
  '>strand_D',
  'GGACUCGGAAACGUGAAGGAGAGGCGCAAGGUUAACCGCCUCAGUCCA',
  '(((((((....))([...[)((((((]........]))))))))))).'],
 ['9BUN',
  '>strand_B',
  'GGACUCGGAAACGUGAAGGAGAGGCGCAAGGUUAACCGCCUCAGUCCA',
  '(((((((....))([...[)((((((]........]))))))))))).'],
 ['9BUN',
  '>strand_C',
  'GGACUCGGAAACGUGAAGGAGAGGCGCAAGGUUAACCGCCUCAGUCCA',
  '(((((((....))([...[)((((((]........]))))))))))).'],
 ['3DIL',
  '>strand_A',
  'gGCCGACGGAGGCGCGCCCGAGAUGAGUAGGCUGUCCCAUCAGGGGAGGAAUCGGGGACGGCUGAAAGGCGAGGGCGCCGAAGGGUGCAGAGUUCCUCCCGCUCUGCAUGCCUGGG

In [33]:
pd.DataFrame(rna_aptamers_predflist, columns=['PDB', 'Strand', 'Sequence', 'Secondary']).to_csv('NAKB_rawRNA.csv', index=False)

In [26]:
#later for bpRNA
rnatypes = {'/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[1315]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[1380]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2154]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2155]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2156]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2252]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2253]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2266]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2267]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2277]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2278]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2284]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2285]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2286]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2287]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2288]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2326]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2327]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2328]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2329]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2330]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2335]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2339]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2354]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2355]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2356]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2357]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2362]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2363]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2504]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2505]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2506]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2526]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2527]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2528]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2529]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2530]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2541]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2542]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2560]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2561]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2562]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2563]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2564]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2588]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2589]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2597]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2600]', '/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[2601]','/html/body/div/form/div[1]/div/fieldset/div/div[3]/span[1]/select/option[26]'}

len(rnatypes)

50

In [34]:
#refinement of ds aptamers
def openedbracket(s):
    br1 = s.count('(')
    br1_ = s.count(')')

    br2 = s.count('{')
    br2_ = s.count('}')

    br3 = s.count('[')
    br3_ = s.count(']')

    br4 = s.count('<')
    br4_ = s.count('>')

    if (br1 == br1_) and (br2 == br2_) and (br3 == br3_) and (br4 == br4_):
        return False
    else:
        return True


refined = []
i =0
while i<len(rna_aptamers_predflist):
    if openedbracket(rna_aptamers_predflist[i][-1]):
        element = [rna_aptamers_predflist[i][0], 'dsRNA', rna_aptamers_predflist[i][2]+rna_aptamers_predflist[i+1][2], rna_aptamers_predflist[i][3]+rna_aptamers_predflist[i+1][3]]
        refined.append(element) 
        i+=1
        
    else:
        element = [rna_aptamers_predflist[i][0], 'ssRNA', rna_aptamers_predflist[i][2], rna_aptamers_predflist[i][3]]
        refined.append(element)

    i+=1
        


In [35]:
len(refined)

628

In [36]:

newrefined = []
i =0
while i<len(refined):
    if openedbracket(refined[i][-1]):
        element = [refined[i][0], 'dsRNA', refined[i][2]+refined[i+1][2], refined[i][3]+refined[i+1][3]]
        newrefined.append(element) 
        i+=1
        
    else:
        element = [refined[i][0], refined[i][1], refined[i][2], refined[i][3]]
        newrefined.append(element)

    i+=1

len(newrefined)

615

In [37]:

newnewrefined = []
i =0
while i<len(newrefined):
    if openedbracket(newrefined[i][-1]):
        element = [newrefined[i][0], 'dsRNA', newrefined[i][2]+newrefined[i+1][2], newrefined[i][3]+newrefined[i+1][3]]
        newnewrefined.append(element) 
        i+=1
        
    else:
        element = [newrefined[i][0], newrefined[i][1], newrefined[i][2], newrefined[i][3]]
        newnewrefined.append(element)

    i+=1

len(newnewrefined)

612

In [57]:
final = []
for a in newnewrefined:
    if len(a[2]) > 5:
        final.append(a)




final = [[f[0], f[1], f[2].upper(), f[3].replace('-', '.')] for f in final]
outtable = pd.DataFrame(final, columns=['PDB','Type', 'Sequence', '2D']).drop_duplicates()
print(len(outtable))
outtable.head()



479


,PDB,Type,Sequence,2D
0,8GXC,dsRNA,GGAGCGUUACGUCCGAAAGUCGCAUUGCACUCCGCGACACGGCUCU...,((((((...[[[[[....(((((..........))))).).)))))...
1,9BUN,ssRNA,GGACUCGGAAACGUGAAGGAGAGGCGCAAGGUUAACCGCCUCAGUCCA,(((((((....))([...[)((((((]........]))))))))))).
5,3DIL,ssRNA,GGCCGACGGAGGCGCGCCCGAGAUGAGUAGGCUGUCCCAUCAGGGG...,(((((((((..(.((((((..(.......(((((((((..(..[[[...
6,8FZA,ssRNA,UUGGGUUCCCUCACCCCAAUCAUAAAAAGG,.((((...[[[...)))).........]]]
8,4Y1I,dsRNA,AAAGGGGAGUAGCGUCGGGAAACCGAAACAAAGUCGUCAAUUCGUG...,(((((......((.((((....))))((((((((((.......(((...


In [58]:
outtable.to_csv('NAKB_RNA.csv', index=False)

In [60]:
#missed pdbids
missed_pdbids = []

for pid in NAKB_pdbids:
    if pid not in list(outtable['PDB']):
        print(pid)
        missed_pdbids.append(pid)


#on these pdbids web server experienced technical difficulties, so if its needed we should obtain them manually


4XBF
8GXB
5FJC
6YMK
6YLB
6YL5
6DME
3DIQ
3UZT
6DB9
6GZR
6YMJ
4TZZ
4YB0
6YMI
3SKW
7KVV
5ELH
2EEW


<h1>all RNA table</h1>

In [74]:
nakb = pd.read_csv('NAKB_RNA.csv')
rnacentre = pd.read_csv('RIBOCENTRE.csv')

rnadb = pd.concat([nakb, rnacentre], axis=0).drop_duplicates()

rnadb.to_csv('RNA_NKAB_AND_RIBOCENTRE.csv')
print(len(rnadb))

rnadb.drop_duplicates('Sequence').to_csv('RNA_NKAB_AND_RIBOCENTRE_NONREDUNDANT.csv')
print(len(rnadb.drop_duplicates('Sequence')))


603
320
